<a href="https://colab.research.google.com/github/alzoqm/cuda_study/blob/main/simple_test_with_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Tue_Mar__8_18:18:20_PST_2022
Cuda compilation tools, release 11.6, V11.6.124
Build cuda_11.6.r11.6/compiler.31057947_0


In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-zadmi94d
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-zadmi94d
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4304 sha256=2e45920340147a9f751701e2261307b4a180cdabd7a01f7782d76f5cc3243b76
  Stored in directory: /tmp/pip-ephem-wheel-cache-p5lw4ikh/wheels/f3/08/cc/e2b5b0e1c92df07dbb50a6f024a68ce090f5e7b2316b41756d
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


# cuda test with colab

In [ ]:
%%cu
#ifndef _TENSOR_H_
#define _TENSOR_H_

#include <cuda_runtime.h>
#include <stdlib.h>
#include <iostream>
#include <stdio.h>
#include <time.h>
#include <unistd.h>

using namespace std;

typedef struct BitField{
    unsigned char a: 1;
    unsigned char b: 1;
    unsigned char c: 1;
    unsigned char d: 1;
    unsigned char e: 1;
    unsigned char f: 1;
    unsigned char g: 1;
    unsigned char h: 1;
}bit_field;

template <typename T>
class Tensor{
public:
    T *value; // Pointer to the data stored in the tensor
    T *temp_value; // Pointer to a temporary storage for the data in the tensor
    unsigned short *tensor_shape; // Array of unsigned short values that represent the shape of the tensor
    unsigned int n; // Number of rows in the tensor
    unsigned int m; // Number of columns in the tensor
    char dim; // Character that represents the number of dimensions of the tensor
    unsigned int sum_size; // Integer that represents the sum of the size of the tensor
    bit_field *mask; // Later, implement the relevant function
    bool is_cuda; // Boolean flag that indicates whether the tensor is stored on a GPU or not
public:
    Tensor(unsigned short *shape, char dim) // Constructor for the Tensor class
    {
        this->dim = dim; // Assign the number of dimensions

        // Allocate memory for the 'tensor_shape' array and store the shape information
        this->tensor_shape = new unsigned short[dim];
        memcpy(this->tensor_shape, shape, sizeof(ushort)*dim);

        // Calculate the total size of the tensor by multiplying the shape information
        this->sum_size = 1;
        for(int i=0; i<dim; i++)
        {
            this->sum_size = this->sum_size*shape[i];
        }
        
        // Calculate the number of rows and columns in the tensor
        this->m = this->tensor_shape[this->dim-1];
        this->n = this->sum_size / this->m;
        
        // Allocate memory for the 'value' array and fill it with random values
        this->value = new T[this->sum_size];
        for(int i=0; i<this->sum_size; ++i)
        {
            value[i] = (((T)rand())/RAND_MAX) + 0.5; // 0~1 range
        }

        // Set the 'is_cuda' member to false (indicating that the tensor is not stored on a GPU)
        this->is_cuda = false;
    }

    Tensor(T *value, unsigned short *shape, char dim) // Constructor allocation Value to Tensor
    {
        this->dim = dim; // Assign the number of dimensions

        // Allocate memory for the 'tensor_shape' array and store the shape information
        this->tensor_shape = new unsigned short[dim];
        memcpy(this->tensor_shape, shape, sizeof(ushort)*dim);
        
        // Calculate the total size of the tensor by multiplying the shape information
        this->sum_size = 1;
        for(int i=0; i<dim; i++)
        {
            this->sum_size = this->sum_size*shape[i];
        }
        
        // Calculate the number of rows and columns in the tensor
        this->m = this->tensor_shape[this->dim-1];
        this->n = this->sum_size / this->m;
        
        // Allocate memory for the 'value' array and copy the values from the input 'value' argument
        this->value = new T[this->sum_size];
        memcpy(this->value, value, this->sum_size*sizeof(T));

        // Set the 'is_cuda' member to false (indicating that the tensor is not stored on a GPU)
        this->is_cuda = false;
    }

    Tensor(T value, unsigned short *shape, char dim) // Constructor allocation Value to Tensor
    {
        this->dim = dim; // Assign the number of dimensions
        // Allocate memory for the 'tensor_shape' array and store the shape information
        this->tensor_shape = new unsigned short[dim];
        memcpy(this->tensor_shape, shape, sizeof(ushort)*dim);
        
        // Calculate the total size of the tensor by multiplying the shape information
        this->sum_size = 1;
        for(int i=0; i<dim; i++)
        {
            this->sum_size = this->sum_size*shape[i];
        }
        
        // Calculate the number of rows and columns in the tensor
        this->m = this->tensor_shape[this->dim-1];
        this->n = this->sum_size / this->m;
        
        // Allocate memory for the 'value' and copy the values from the input 'value' argument
        this->value = new T[this->sum_size];
        for(int i=0; i<this->sum_size; i++)
        {
            this->value[i] = value;
        }
        //memcpy(this->value, value, this->sum_size*sizeof(T));

        // Set the 'is_cuda' member to false (indicating that the tensor is not stored on a GPU)
        this->is_cuda = false;
    }

    ~Tensor() // Destructor for the Tensor class
    {   
        if(this->is_cuda==true) // Check if the tensor is stored on a GPU or in CPU memory
        {
            cudaFree(this->value);
        }
        else // else in cpu
        {
            delete this->value;
        }
    }

    unsigned short *shape() // Returns the shape of the Tensor2D as an array of unsigned short integers
    {
        // Allocate memory for a temporary shape array
        unsigned short *temp_shape;
        temp_shape = new unsigned short[this->dim];

        // Copy the shape from 'tensor_shape' to 'temp_shape'
        memcpy(temp_shape, this->tensor_shape, this->dim);

        // Return the temporary shape array
        return temp_shape;
    }

    T *return_value() // Returns the value of the Tensor2D as an array of template
    {
        if(this->is_cuda==true) // Check if the tensor is stored on a GPU or in CPU memory
        {
            this->cpu(); // If stored on a GPU, transfer it to CPU memory
            T *value = new T[this->sum_size]; // Allocate memory for a temporary value array
            memcpy(value, this->value, this->sum_size*sizeof(T)); // Copy the value from 'this->value' to 'value'
            this->cuda(); // Transfer the tensor back to GPU memory
            return value; // Return the temporary value array
        }
        else
        {
            T *value = new T[this->sum_size]; // If stored in CPU memory, allocate memory for a temporary value array
            memcpy(value, this->value, this->sum_size*sizeof(T)); // Copy the value from 'this->value' to 'value'
            return value; // Return the temporary value array
        }
    }

    void print()
    {   
        if(this->is_cuda==true)
        {
            this->cpu();
            printf("(value: \n");
            unsigned int *check_size = new unsigned int[this->dim+1];
            for (unsigned int i = 0; i <= this->dim; i++) 
            {
                check_size[i] = 1;
            }
            for(int i=this->dim-1; i>=0; i--)
            {
                check_size[i] = this->tensor_shape[i] * check_size[i+1];
            }
            if(this->n <= 6)
            {
                for(int i=0; i<this->n; i++)
                {
                    if(this->m <= 6)
                    {
                        for(int j=0; j<this->m; j++)
                        {
                            for(int k=0; k<this->dim; k++)
                            {
                                if((i*this->m+j)%check_size[k]==0)
                                {
                                    printf("[");
                                }
                            }
                            cout<<value[i*this->m+j];
                            if(j != this->m-1)
                            {
                                printf(", ");
                            }
                            for(int k=0; k<this->dim; k++)
                            {
                                if((i*this->m+j)%check_size[k] == check_size[k]-1)
                                {
                                    printf("]");
                                }
                            }
                        }
                        printf("\n");
                    }
                    else
                    {
                        for(int j=0; j<3; j++)
                        {
                            for(int k=0; k<this->dim; k++)
                            {
                                if((i*this->m+j)%check_size[k]==0)
                                {
                                    printf("[");
                                }
                            }
                            cout<<value[i*this->m+j];
                            printf(", ");
                        }
                        printf("..... ");
                        for(int j=this->m-3; j<this->m; j++)
                        {
                            cout<<value[i*this->m+j];
                            if(j != this->m-1)
                            {
                                printf(", ");
                            }
                            for(int k=0; k<this->dim; k++)
                            {
                                if((i*this->m+j)%check_size[k] == check_size[k]-1)
                                {
                                    printf("]");
                                }
                            }
                        }
                        printf("\n");
                    }
                }
            }
            else
            {
                for(int i=0; i<3; i++)
                {
                    if(this->m <= 6)
                    {
                        for(int j=0; j<this->m; j++)
                        {
                            for(int k=0; k<this->dim; k++)
                            {
                                if((i*this->m+j)%check_size[k]==0)
                                {
                                    printf("[");
                                }
                            }
                            cout<<value[i*this->m+j];
                            if(j != this->m-1)
                            {
                                printf(", ");
                            }
                            for(int k=0; k<this->dim; k++)
                            {
                                if((i*this->m+j)%check_size[k] == check_size[k]-1)
                                {
                                    printf("]");
                                }
                            }
                        }
                        printf("\n");
                    }
                    else
                    {
                        for(int j=0; j<3; j++)
                        {
                            for(int k=0; k<this->dim; k++)
                            {
                                if((i*this->m+j)%check_size[k]==0)
                                {
                                    printf("[");
                                }
                            }
                            cout<<value[i*this->m+j];
                            printf(", ");
                        }
                        printf("..... ");
                        for(int j=this->m-3; j<this->m; j++)
                        {
                            cout<<value[i*this->m+j];
                            if(j != this->m-1)
                            {
                                printf(", ");
                            }
                            for(int k=0; k<this->dim; k++)
                            {
                                if((i*this->m+j)%check_size[k] == check_size[k]-1)
                                {
                                    printf("]");
                                }
                            }
                        }
                        printf("\n");
                    }
                }
                printf(". . .\n");
                printf(". . .\n");
                for(int i=this->n-3; i<this->n; i++)
                {
                    if(this->m <= 6)
                    {
                        for(int j=0; j<this->m; j++)
                        {
                            for(int k=0; k<this->dim; k++)
                            {
                                if((i*this->m+j)%check_size[k]==0)
                                {
                                    printf("[");
                                }
                            }
                            cout<<value[i*this->m+j];
                            if(j != this->m-1)
                            {
                                printf(", ");
                            }
                            for(int k=0; k<this->dim; k++)
                            {
                                if((i*this->m+j)%check_size[k] == check_size[k]-1)
                                {
                                    printf("]");
                                }
                            }
                        }
                        printf("\n");
                    }
                    else
                    {
                        for(int j=0; j<3; j++)
                        {
                            for(int k=0; k<this->dim; k++)
                            {
                                if((i*this->m+j)%check_size[k]==0)
                                {
                                    printf("[");
                                }
                            }
                            cout<<value[i*this->m+j];
                            printf(", ");
                        }
                        printf("....., ");
                        for(int j=this->m-3; j<this->m; j++)
                        {
                            cout<<value[i*this->m+j];
                            if(j != this->m-1)
                            {
                                printf(", ");
                            }
                            for(int k=0; k<this->dim; k++)
                            {
                                if((i*this->m+j)%check_size[k] == check_size[k]-1)
                                {
                                    printf("]");
                                }
                            }
                        }
                        printf("\n");
                    }
                }
            }
            printf("shape: (");
            for(int i=0; i<this->dim; i++)
            {
                printf("%d", this->tensor_shape[i]);
                if(i!=this->dim-1)
                {
                    printf(", ");
                }
            }
            printf(")\n");
            this->cuda();
            cout<<"is_cuda: "<<this->is_cuda<<")\n";
        }
        else
        {
            printf("(value: \n");
            unsigned int *check_size = new unsigned int[this->dim+1];
            for (unsigned int i = 0; i <= this->dim; i++) 
            {
                check_size[i] = 1;
            }
            for(int i=this->dim-1; i>=0; i--)
            {
                check_size[i] = this->tensor_shape[i] * check_size[i+1];
            }
            if(this->n <= 6)
            {
                for(int i=0; i<this->n; i++)
                {
                    if(this->m <= 6)
                    {
                        for(int j=0; j<this->m; j++)
                        {
                            for(int k=0; k<this->dim; k++)
                            {
                                if((i*this->m+j)%check_size[k]==0)
                                {
                                    printf("[");
                                }
                            }
                            cout<<value[i*this->m+j];
                            if(j != this->m-1)
                            {
                                printf(", ");
                            }
                            for(int k=0; k<this->dim; k++)
                            {
                                if((i*this->m+j)%check_size[k] == check_size[k]-1)
                                {
                                    printf("]");
                                }
                            }
                        }
                        printf("\n");
                    }
                    else
                    {
                        for(int j=0; j<3; j++)
                        {
                            for(int k=0; k<this->dim; k++)
                            {
                                if((i*this->m+j)%check_size[k]==0)
                                {
                                    printf("[");
                                }
                            }
                            cout<<value[i*this->m+j];
                            printf(", ");
                        }
                        printf("....., ");
                        for(int j=this->m-3; j<this->m; j++)
                        {
                            cout<<value[i*this->m+j];
                            if(j != this->m-1)
                            {
                                printf(", ");
                            }
                            for(int k=0; k<this->dim; k++)
                            {
                                if((i*this->m+j)%check_size[k] == check_size[k]-1)
                                {
                                    printf("]");
                                }
                            }
                        }
                        printf("\n");
                    }
                }
            }
            else
            {
                for(int i=0; i<3; i++)
                {
                    if(this->m <= 6)
                    {
                        for(int j=0; j<this->m; j++)
                        {
                            for(int k=0; k<this->dim; k++)
                            {
                                if((i*this->m+j)%check_size[k]==0)
                                {
                                    printf("[");
                                }
                            }
                            cout<<value[i*this->m+j];
                            if(j != this->m-1)
                            {
                                printf(", ");
                            }
                            for(int k=0; k<this->dim; k++)
                            {
                                if((i*this->m+j)%check_size[k] == check_size[k]-1)
                                {
                                    printf("]");
                                }
                            }
                        }
                        printf("\n");
                    }
                    else
                    {
                        for(int j=0; j<3; j++)
                        {
                            for(int k=0; k<this->dim; k++)
                            {
                                if((i*this->m+j)%check_size[k]==0)
                                {
                                    printf("[");
                                }
                            }
                            cout<<value[i*this->m+j];
                            printf(", ");
                        }
                        printf("....., ");
                        for(int j=this->m-3; j<this->m; j++)
                        {
                            cout<<value[i*this->m+j];
                            if(j != this->m-1)
                            {
                                printf(", ");
                            }
                            for(int k=0; k<this->dim; k++)
                            {
                                if((i*this->m+j)%check_size[k] == check_size[k]-1)
                                {
                                    printf("]");
                                }
                            }
                        }
                        printf("\n");
                    }
                }
                printf(". . .\n");
                printf(". . .\n");
                for(int i=this->n-3; i<this->n; i++)
                {
                    if(this->m <= 6)
                    {
                        for(int j=0; j<this->m; j++)
                        {
                            for(int k=0; k<this->dim; k++)
                            {
                                if((i*this->m+j)%check_size[k]==0)
                                {
                                    printf("[");
                                }
                            }
                            cout<<value[i*this->m+j];
                            if(j != this->m-1)
                            {
                                printf(", ");
                            }
                            for(int k=0; k<this->dim; k++)
                            {
                                if((i*this->m+j)%check_size[k] == check_size[k]-1)
                                {
                                    printf("]");
                                }
                            }
                        }
                        printf("\n");
                    }
                    else
                    {
                        for(int j=0; j<3; j++)
                        {
                            for(int k=0; k<this->dim; k++)
                            {
                                if((i*this->m+j)%check_size[k]==0)
                                {
                                    printf("[");
                                }
                            }
                            cout<<value[i*this->m+j];
                            printf(", ");
                        }
                        printf("....., ");
                        for(int j=this->m-3; j<this->m; j++)
                        {
                            cout<<value[i*this->m+j];
                            if(j != this->m-1)
                            {
                                printf(", ");
                            }
                            for(int k=0; k<this->dim; k++)
                            {
                                if((i*this->m+j)%check_size[k] == check_size[k]-1)
                                {
                                    printf("]");
                                }
                            }
                        }
                        printf("\n");
                    }
                }
            }
            printf("shape: (");
            for(int i=0; i<this->dim; i++)
            {
                printf("%d", this->tensor_shape[i]);
                if(i!=this->dim-1)
                {
                    printf(", ");
                }
            }
            printf(")\n");
            cout<<"is_cuda: "<<this->is_cuda<<")\n";
        }
    }

    void cuda() // Copy the tensor data from the CPU memory to the GPU memory.
    {
        if(this->is_cuda==true)
        {
            return;
        }
        // Allocate temporary memory on the GPU to store the tensor data
        cudaMalloc((void**)&this->temp_value, sizeof(T)*this->sum_size); 
        // Copy the tensor data from the CPU memory to the temporary GPU memory
        cudaMemcpy(this->temp_value, this->value, sizeof(T)*this->sum_size, cudaMemcpyHostToDevice);
        delete this->value; // Delete the original tensor data in the CPU memory

        // Allocate memory on the GPU to store the tensor data
        cudaMalloc((void**)&this->value, sizeof(T)*this->sum_size);
        // Copy the tensor data from the temporary GPU memory to the GPU memory
        cudaMemcpy(this->value, this->temp_value, sizeof(T)*this->sum_size, cudaMemcpyDeviceToDevice);
        // Free the temporary GPU memory
        cudaFree(this->temp_value);
        // Update the flag to indicate that the tensor data is now stored on the GPU
        this->is_cuda = true;
    }

    void cpu() // allocate the tensor data from the GPU memory to the CPU memory.
    {
        if(this->is_cuda==false)
        {
            return;
        }
        // Allocate temporary memory on the CPU to store the tensor data
        this->temp_value = new T[this->sum_size];
        // Copy the tensor data from the GPU memory to the temporary CPU memory
        cudaMemcpy(this->temp_value, this->value, sizeof(T)*this->sum_size, cudaMemcpyDeviceToHost);
        cudaFree(this->value); // Free the GPU memory

        // Allocate memory on the CPU to store the tensor data
        this->value = new T[this->sum_size];
        // Copy the tensor data from the temporary CPU memory to the CPU memory
        memcpy(this->value, this->temp_value, this->sum_size*sizeof(T));
        // Delete the temporary CPU memory
        delete this->temp_value;
        // Update the flag to indicate that the tensor data is now stored on the CPU
        this->is_cuda = false;
    }
};

#endif

template <typename T>
__global__ void Matrix_Add(T *d_a, T *d_b, T *d_c, unsigned int n, unsigned int m) 
{
  int i = blockIdx.x * blockDim.x + threadIdx.x;
  int j = blockIdx.y * blockDim.y + threadIdx.y;
  if (i < n && j < m) 
  {
    d_c[i * m + j] = d_a[i * m + j] + d_b[i * m + j];
  }
}

template <typename T>
__global__ void Matrix_Sub(T *d_a, T *d_b, T *d_c, unsigned int n, unsigned int m) 
{
  int i = blockIdx.x * blockDim.x + threadIdx.x;
  int j = blockIdx.y * blockDim.y + threadIdx.y;
  if (i < n && j < m) 
  {
    d_c[i * m + j] = d_a[i * m + j] - d_b[i * m + j];
  }
}

template <typename T>
__global__ void Matrix_Div(T *d_a, T *d_b, T *d_c, unsigned int n, unsigned int m) 
{
  int i = blockIdx.x * blockDim.x + threadIdx.x;
  int j = blockIdx.y * blockDim.y + threadIdx.y;
  if (i < n && j < m) 
  {
    d_c[i * m + j] = d_a[i * m + j] / d_b[i * m + j];
  }
}

template <typename T>
__global__ void MatrixMul(T *d_a, T *d_b, T *d_c, unsigned int n, unsigned int m) // normal multiplication
{ 
  int i = blockIdx.x * blockDim.x + threadIdx.x;
  int j = blockIdx.y * blockDim.y + threadIdx.y;
  if (i < n && j < m) 
  {
    d_c[i * m + j] = d_a[i * m + j] * d_b[i * m + j];
  }
}

template <typename T>
__global__ void Matrix_Mul(T *a,T *b, T *c, unsigned int bs, unsigned int n, unsigned int m, unsigned int p)
{ 
    int l = blockIdx.x * blockDim.x + threadIdx.x;
    int i = blockIdx.y * blockDim.y + threadIdx.y; 
    int j = blockIdx.z * blockDim.z + threadIdx.z;
    
    if(l < bs && i < n && j < p ) 
    {
        T sum = 0.;
        for(int k = 0; k < m; k++) 
        {
            //sum += a[batch_size*n*m + row * m + i] * b[batch_size*m*p + i * p + col];
            sum += a[l*n*m + i * m + k] * b[l*m*p + k * p + j];
        }
        c[l*n*p + i * p + j] = sum;
    }
}

template <typename T>
void cpu_Matrix_Add(T &a, T &b, T &c)
{
    for(int i=0; i<c.n; i++)
    {
        for(int j=0; j<c.m; j++)
        {
            c.value[i*c.m + j] = a.value[i*c.m + j] + b.value[i*c.m + j];
        }
    }
}

template <typename T>
void cpu_Matrix_Sub(T &a, T &b, T &c)
{
    for(int i=0; i<c.n; i++)
    {
        for(int j=0; j<c.m; j++)
        {
            c.value[i*c.m + j] = a.value[i*c.m + j] - b.value[i*c.m + j];
        }
    }
}

template <typename T>
void cpu_Matrix_Div(T &a, T &b, T &c)
{
    for(int i=0; i<c.n; i++)
    {
        for(int j=0; j<c.m; j++)
        {
            c.value[i*c.m + j] = a.value[i*c.m + j] / b.value[i*c.m + j];
        }
    }
}

template <typename T>
void cpu_MatrixMul(T &a, T &b, T &c)
{
    for(int i=0; i<c.n; i++)
    {
        for(int j=0; j<c.m; j++)
        {
            c.value[i*c.m + j] = a.value[i*c.m + j] * b.value[i*c.m + j];
        }
    }
}


template <typename T>
void cpu_Matrix_Mul(T *a, T *b, T *c, unsigned short bs, unsigned short n, unsigned short m, unsigned short p)
{
    for(int l=0; l<bs; l++)
    {
        for (int i = 0; i < n; ++i) 
        {
            for (int j = 0; j < p; ++j) 
            {
                T sum = 0.;
                for (int k = 0; k < m; ++k) 
                {
                    sum += a[l*n*m + i * m + k] * b[l*m*p + k * p + j];
                }
                c[l*n*p + i * p + j] = sum;
            }
        }
    }
}

template <typename T>
Tensor<T> matadd(Tensor<T> &a, Tensor<T> &b)
{
    if(a.dim != b.dim)
    {
        throw "tensor1 and tensor2 dim is not same!\n";
    }
    for (int i = 0; i < a.dim; i++) 
    {
        if (a.tensor_shape[i] != b.tensor_shape[i]) 
        {
            throw invalid_argument("tensor1 and tensor2 shape is not same!");
        }
    }
    if(a.is_cuda != b.is_cuda)
    {
        if(a.is_cuda==true)
        {
            throw "tensor1 is cuda but tensor2 is cpu.\n";
        }
        else
        {
            throw "tensor1 is cpu but tensor2 is cuda.\n";
        }
    }
    unsigned short c_shape[a.dim];
    memcpy(c_shape, a.tensor_shape, sizeof(short)*a.dim);
    Tensor<T> c(c_shape, a.dim);
    if(a.is_cuda==true)
    {
        c.cuda();
        dim3 block(16, 16);
        dim3 grid((c.n + block.x - 1) / block.x, (c.m + block.y - 1) / block.y);
        Matrix_Add<T><<<grid, block>>>(a.value, b.value, c.value, c.n, c.m);
        return c;
    }
    else
    {
        cpu_Matrix_Add(a, b, c);
        return c;
    }
}

template <typename T>
void matadd(Tensor<T> &a, Tensor<T> &b, Tensor<T> &c)
{
    if(a.dim != b.dim)
    {
        throw "tensor1 and tensor2 dim is not same!\n";
    }
    if(a.dim != c.dim)
    {
        throw "input and output dim is not same!\n";
    }
    for (int i = 0; i < a.dim; i++) 
    {
        if (a.tensor_shape[i] != b.tensor_shape[i]) 
        {
            throw invalid_argument("tensor1 and tensor2 shape is not same!");
        }
        if(a.tensor_shape[i] != c.tensor_shape[i])
        {
            throw invalid_argument("input shape and output shape is not same!");
        }
    }
    if(a.is_cuda != b.is_cuda)
    {
        if(a.is_cuda==true)
        {
            throw "tensor1 is cuda but tensor2 is cpu.\n";
        }
        else
        {
            throw "tensor1 is cpu but tensor2 is cuda.\n";
        }
    }
    if(a.is_cuda != c.is_cuda)
    {
        if(a.is_cuda==true)
        {
            throw "input is cuda but output is cpu.\n";
        }
        else
        {
            throw "input is cpu but output is cuda.\n";
        }
    }

    if(a.is_cuda==true)
    {
        dim3 block(16, 16);
        dim3 grid((c.n + block.x - 1) / block.x, (c.m + block.y - 1) / block.y);
        Matrix_Add<T><<<grid, block>>>(a.value, b.value, c.value, c.n, c.m);
    }
    else
    {
        cpu_Matrix_Add(a, b, c);
    }
}

template <typename T>
Tensor<T> matsub(Tensor<T> &a, Tensor<T> &b)
{
    if(a.dim != b.dim)
    {
        throw "tensor1 and tensor2 dim is not same!\n";
    }
    for (int i = 0; i < a.dim; i++) 
    {
        if (a.tensor_shape[i] != b.tensor_shape[i]) 
        {
            throw invalid_argument("tensor1 and tensor2 shape is not same!");
        }
    }
    if(a.is_cuda != b.is_cuda)
    {
        if(a.is_cuda==true)
        {
            throw "tensor1 is cuda but tensor2 is cpu.\n";
        }
        else
        {
            throw "tensor1 is cpu but tensor2 is cuda.\n";
        }
    }
    unsigned short c_shape[a.dim];
    memcpy(c_shape, a.tensor_shape, sizeof(short)*a.dim);
    Tensor<T> c(c_shape, a.dim);
    if(a.is_cuda==true)
    {
        c.cuda(); 
        dim3 block(16, 16);
        dim3 grid((c.n + block.x - 1) / block.x, (c.m + block.y - 1) / block.y);
        Matrix_Sub<T><<<grid, block>>>(a.value, b.value, c.value, c.n, c.m);
        return c;
    }
    else
    {
        cpu_Matrix_Sub(a, b, c);
        return c;
    }
}

template <typename T>
void matsub(Tensor<T> &a, Tensor<T> &b, Tensor<T> &c)
{
    if(a.dim != b.dim)
    {
        throw "tensor1 and tensor2 dim is not same!\n";
    }
    if(a.dim != c.dim)
    {
        throw "input and output dim is not same!\n";
    }
    for (int i = 0; i < a.dim; i++) 
    {
        if (a.tensor_shape[i] != b.tensor_shape[i]) 
        {
            throw invalid_argument("tensor1 and tensor2 shape is not same!");
        }
        if(a.tensor_shape[i] != c.tensor_shape[i])
        {
            throw invalid_argument("input shape and output shape is not same!");
        }
    }
    if(a.is_cuda != b.is_cuda)
    {
        if(a.is_cuda==true)
        {
            throw "tensor1 is cuda but tensor2 is cpu.\n";
        }
        else
        {
            throw "tensor1 is cpu but tensor2 is cuda.\n";
        }
    }
    if(a.is_cuda != c.is_cuda)
    {
        if(a.is_cuda==true)
        {
            throw "input is cuda but output is cpu.\n";
        }
        else
        {
            throw "input is cpu but output is cuda.\n";
        }
    }
    if(a.is_cuda==true)
    {
        dim3 block(16, 16);
        dim3 grid((c.n + block.x - 1) / block.x, (c.m + block.y - 1) / block.y);
        Matrix_Sub<T><<<grid, block>>>(a.value, b.value, c.value, c.n, c.m);
    }
    else
    {
        cpu_Matrix_Sub(a, b, c);
    }
}

template <typename T>
Tensor<T> matdiv(Tensor<T> &a, Tensor<T> &b)
{
    if(a.dim != b.dim)
    {
        throw "tensor1 and tensor2 dim is not same!\n";
    }
    for (int i = 0; i < a.dim; i++) 
    {
        if (a.tensor_shape[i] != b.tensor_shape[i]) 
        {
            throw invalid_argument("tensor1 and tensor2 shape is not same!");
        }
    }
    if(a.is_cuda != b.is_cuda)
    {
        if(a.is_cuda==true)
        {
            throw "tensor1 is cuda but tensor2 is cpu.\n";
        }
        else
        {
            throw "tensor1 is cpu but tensor2 is cuda.\n";
        }
    }
    unsigned short c_shape[a.dim];
    memcpy(c_shape, a.tensor_shape, sizeof(short)*a.dim);
    Tensor<T> c(c_shape, a.dim);
    if(a.is_cuda==true)
    {
        c.cuda();
        dim3 block(16, 16);
        dim3 grid((c.n + block.x - 1) / block.x, (c.m + block.y - 1) / block.y);
        Matrix_Div<T><<<grid, block>>>(a.value, b.value, c.value, c.n, c.m);
        return c;
    }
    else
    {
        cpu_Matrix_Div(a, b, c);
        return c;
    }
}

template <typename T>
void matdiv(Tensor<T> &a, Tensor<T> &b, Tensor<T> &c)
{
    if(a.dim != b.dim)
    {
        throw "tensor1 and tensor2 dim is not same!\n";
    }
    if(a.dim != c.dim)
    {
        throw "input and output dim is not same!\n";
    }
    for (int i = 0; i < a.dim; i++) 
    {
        if (a.tensor_shape[i] != b.tensor_shape[i]) 
        {
            throw invalid_argument("tensor1 and tensor2 shape is not same!");
        }
        if(a.tensor_shape[i] != c.tensor_shape[i])
        {
            throw invalid_argument("input shape and output shape is not same!");
        }
    }
    if(a.is_cuda != b.is_cuda)
    {
        if(a.is_cuda==true)
        {
            throw "tensor1 is cuda but tensor2 is cpu.\n";
        }
        else
        {
            throw "tensor1 is cpu but tensor2 is cuda.\n";
        }
    }
    if(a.is_cuda != c.is_cuda)
    {
        if(a.is_cuda==true)
        {
            throw "input is cuda but output is cpu.\n";
        }
        else
        {
            throw "input is cpu but output is cuda.\n";
        }
    }
    if(a.is_cuda==true)
    {
        dim3 block(16, 16);
        dim3 grid((c.n + block.x - 1) / block.x, (c.m + block.y - 1) / block.y);
        Matrix_Div<T><<<grid, block>>>(a.value, b.value, c.value, c.n, c.m);
    }
    else
    {
        cpu_Matrix_Div(a, b, c);
    }
}

template <typename T>
Tensor<T> matmul(Tensor<T> &a, Tensor<T> &b)
{
    if(a.dim != b.dim)
    {
        throw "tensor1 and tensor2 dim is not same!\n";
    }
    for (int i = 0; i < a.dim; i++) 
    {
        if (a.tensor_shape[i] != b.tensor_shape[i]) 
        {
            throw invalid_argument("tensor1 and tensor2 shape is not same!");
        }
    }
    if(a.is_cuda != b.is_cuda)
    {
        if(a.is_cuda==true)
        {
            throw "tensor1 is cuda but tensor2 is cpu.\n";
        }
        else
        {
            throw "tensor1 is cpu but tensor2 is cuda.\n";
        }
    }
    unsigned short c_shape[a.dim];
    memcpy(c_shape, a.tensor_shape, sizeof(short)*a.dim);
    Tensor<T> c(c_shape, a.dim);
    if(a.is_cuda==true)
    {
        c.cuda();
        dim3 block(16, 16);
        dim3 grid((c.n + block.x - 1) / block.x, (c.m + block.y - 1) / block.y);
        MatrixMul<T><<<grid, block>>>(a.value, b.value, c.value, c.n, c.m);
        return c;
    }
    else
    {
        cpu_MatrixMul(a, b, c);
        return c;
    }
}

template <typename T>
void matmul(Tensor<T> &a, Tensor<T> &b, Tensor<T> &c)
{
    if(a.dim != b.dim)
    {
        throw "tensor1 and tensor2 dim is not same!\n";
    }
    if(a.dim != c.dim)
    {
        throw "input and output dim is not same!\n";
    }
    for (int i = 0; i < a.dim; i++) 
    {
        if (a.tensor_shape[i] != b.tensor_shape[i]) 
        {
            throw invalid_argument("tensor1 and tensor2 shape is not same!");
        }
        if(a.tensor_shape[i] != c.tensor_shape[i])
        {
            throw invalid_argument("input shape and output shape is not same!");
        }
    }
    if(a.is_cuda != b.is_cuda)
    {
        if(a.is_cuda==true)
        {
            throw "tensor1 is cuda but tensor2 is cpu.\n";
        }
        else
        {
            throw "tensor1 is cpu but tensor2 is cuda.\n";
        }
    }
    if(a.is_cuda != c.is_cuda)
    {
        if(a.is_cuda==true)
        {
            throw "input is cuda but output is cpu.\n";
        }
        else
        {
            throw "input is cpu but output is cuda.\n";
        }
    }
    if(a.is_cuda==true)
    {
        dim3 block(16, 16);
        dim3 grid((c.n + block.x - 1) / block.x, (c.m + block.y - 1) / block.y);
        MatrixMul<T><<<grid, block>>>(a.value, b.value, c.value, c.n, c.m);
    }
    else
    {
        cpu_MatrixMul(a, b, c);
    }
}

template <typename T>
Tensor<T> mat_mul(Tensor<T> &a, Tensor<T> &b)
{
    if(a.dim != b.dim)
    {
        throw "tensor1 and tensor2 dim is not same!\n";
    }
    unsigned int b_n = b.tensor_shape[b.dim-2];
    if(a.m != b_n)
    {
        throw "tensor1 col and tensor2 row is not same!\n";
    }
    for(int i=0; i<a.dim-2; i++)
    {
        if(a.tensor_shape[i] != b.tensor_shape[i])
        {
            throw "tensor1 shape and tensor2 shape is not same!\n";
        }
    }
    if(a.is_cuda != b.is_cuda)
    {
        if(a.is_cuda==true)
        {
            throw "tensor1 is cuda but tensor2 is cpu.\n";
        }
        else
        {
            throw "tensor1 is cpu but tensor2 is cuda.\n";
        }
    }
    char c_dim = a.dim;
    unsigned short c_shape[c_dim];
    for(int i=0; i<a.dim-2; i++)
    {
        c_shape[i] = a.tensor_shape[i];
    }
    c_shape[c_dim-2] = a.tensor_shape[c_dim-2];
    c_shape[c_dim-1] = b.tensor_shape[c_dim-1];
    Tensor<T> c(c_shape, c_dim);
    int a_n = a.tensor_shape[a.dim-2];
    int a_b = a.n / a_n;
    int a_m = a.m;
    int b_m = b.m;
    if(a.is_cuda==true)
    {
        c.cuda();
        // dim3 block(16, 16);
        // dim3 grid((c.n + block.x - 1) / block.x, (c.m + block.y - 1) / block.y);
        dim3 block(4, 16, 16);
        dim3 grid((a_b + block.x - 1) / block.x, (a_n + block.y - 1) / block.y, (b_m + block.z - 1) / block.z);
        Matrix_Mul<T><<<grid, block>>>(a.value, b.value, c.value, a_b, a_n, a_m, b_m);
        return c;
    }
    else
    {
        cpu_Matrix_Mul<T>(a.value, b.value, c.value, a_b, a_n, a_m, b_m);
        return c;
    }
}

template <typename T>
void mat_mul(Tensor<T> &a, Tensor<T> &b, Tensor<T> &c)
{
    if(a.dim != b.dim)
    {
        throw "tensor1 and tensor2 dim is not same!\n";
    }
    unsigned int b_n = b.tensor_shape[b.dim-2];
    if(a.m != b_n)
    {
        throw "tensor1 col and tensor2 row is not same!\n";
    }
    for(int i=0; i<a.dim-2; i++)
    {
        if(a.tensor_shape[i] != b.tensor_shape[i])
        {
            throw "tensor1 shape and tensor2 shape is not same!\n";
        }
    }
    if(a.is_cuda != b.is_cuda || a.is_cuda != c.is_cuda)
    {
        if(a.is_cuda==true)
        {
            throw "tensor1 is cuda but tensor2 is cpu.\n";
        }
        else
        {
            throw "tensor1 is cpu but tensor2 is cuda.\n";
        }
    }
    int a_n = a.tensor_shape[a.dim-2];
    int a_b = a.n / a_n;
    int a_m = a.m;
    int b_m = b.m;
    if(a.is_cuda==true)
    {
        c.cuda();
        dim3 block(4, 16, 16);
        dim3 grid((a_b + block.x - 1) / block.x, (a_n + block.y - 1) / block.y, (b_m + block.z - 1) / block.z);
        Matrix_Mul<T><<<grid, block>>>(a.value, b.value, c.value, a_b, a_n, a_m, b_m);
    }
    else
    {
        cpu_Matrix_Mul<T>(a.value, b.value, c.value, a_b, a_n, a_m, b_m);
    }
}

__host__ int main()
{
    srand(time(NULL));
    unsigned short size1[] = {256, 256, 512};
    unsigned short size2[] = {256, 512, 256};
    unsigned short size3[] = {256, 256, 256};

    Tensor<double> tensor1(size1, sizeof(size1) / sizeof(unsigned short));
    Tensor<double> tensor2(size2, sizeof(size2) / sizeof(unsigned short));
    Tensor<double> tensor3(size3, sizeof(size3) / sizeof(unsigned short));
    Tensor<double> tensor3_cpu(size3, sizeof(size3) / sizeof(unsigned short));

// CPU mat_mul running
    // clock_t cpu_start = clock();
    // mat_mul(tensor1, tensor2, tensor3_cpu);
    // clock_t cpu_end = clock();
    // double cpu_time = (double)(cpu_end - cpu_start) / CLOCKS_PER_SEC;

// Before allocating cuda
    size_t before_free, before_total;
    cudaMemGetInfo(&before_free, &before_total);
    printf("Total GPU memory: %lu MB\nbefore Free GPU memory: %lu MB\n", before_total/1000000, before_free/1000000);

// cuda activate
    clock_t compile_start = clock();
    tensor1.cuda();
    tensor2.cuda();
    tensor3.cuda();
    clock_t compile_end = clock();
    double compile_time = (double)(compile_end - compile_start) / CLOCKS_PER_SEC;

// Check GPU memory usage
    size_t free, total;
    cudaMemGetInfo(&free, &total);
    printf("after Free GPU memory: %lu MB\n", free/1000000);
    printf("diff before after memory: %lu MB\n\n", before_free/1000000- free/1000000);

// GPU mat_mul running
    clock_t gpu_start = clock();
    mat_mul(tensor1, tensor2, tensor3);
    clock_t gpu_end = clock();
    double gpu_time = (double)(gpu_end - gpu_start) / CLOCKS_PER_SEC;

// print time and value
    // printf("cpu matmul time: %f\n", cpu_time);
    printf("GPU compile time: %f\n", compile_time);
    printf("GPU matmul time: %f\n", gpu_time);
    printf("GPU sum time: %f\n\n", compile_time+gpu_time);
    // tensor3_cpu.print();
    tensor3.print();

    return 0;
}

Total GPU memory: 15843 MB
before Free GPU memory: 15738 MB
after Free GPU memory: 15067 MB
diff before after memory: 671 MB

GPU compile time: 0.192638
GPU matmul time: 0.000016
GPU sum time: 0.192654

(value: 
[[[518.396, 501.015, 507.937, ..... 506.403, 497.277, 510.718]
[531.592, 513.589, 518.836, ..... 515.125, 511.093, 525.992]
[511.173, 498.659, 502.862, ..... 499.375, 493.607, 505.634]
. . .
. . .
[516.915, 511.526, 507.239, ....., 509.046, 503.845, 527.401]
[514.958, 514.663, 509.139, ....., 504.465, 504.288, 528.421]
[509.262, 505.555, 500.574, ....., 500.698, 497.363, 523.914]]]
shape: (256, 256, 256)
is_cuda: 1)

